In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# Create a dataset from directories cars_fake and cars_real
from torchvision import datasets, transforms
import os

# Create a dataset from directories cars_fake and cars_real
data_dirs = ['../cars_dataset/cars/1_fake', '../cars_dataset/cars/0_real']

print("Current working directory before: ", os.getcwd())
proj_dir = "/dtu/blackhole/1d/155421/Deeplearning-Group16"
# Set the current working directory to the project directory
os.chdir(proj_dir)
print("Current working directory: ", os.getcwd())

Current working directory before:  /dtu/blackhole/1d/155421/Deeplearning-Group16
Current working directory:  /dtu/blackhole/1d/155421/Deeplearning-Group16


### Models

In [3]:
import torch.nn as nn 
import torch

from resnet50nodown import resnet50nodown

class DIMD(nn.Module):

    def __init__(self):
        super(DIMD, self).__init__()
        self.model = resnet50nodown(num_classes=1)

    def forward(self, x):
        return self.model(x)

    def load_weights(self, ckpt):
        state_dict = torch.load(ckpt, map_location='cpu')
        try:
            self.model.load_state_dict(state_dict['model'])
            if ('module._conv_stem.weight' in state_dict['model']) or ('module.fc.fc1.weight' in state_dict['model']) or ('module.fc.weight' in state_dict['model']):
                self.load_state_dict({key[7:]: state_dict['model'][key] for key in state_dict['model']})
            else:
                self.model.load_state_dict(state_dict['model'])

        except:
            print('Loading state dict failed. Trying to load without model prefix.')
            self.model.load_state_dict(state_dict)

    def predict(self, img):
        with torch.no_grad():
            logits = self.forward(img)
            return logits.sigmoid().flatten().tolist()

In [4]:
import clip 
import torch.nn as nn 
import torch

class UnivFD(nn.Module):
    def __init__(self, num_classes=1):
        super(UnivFD, self).__init__()
        self.model, self.preprocess = clip.load("ViT-L/14", device="cpu")
        self.fc = nn.Linear(768, num_classes)

    def forward(self, x):
        features = self.model.encode_image(x) 
        return self.fc(features)
    
    def load_weights(self, ckpt):
        state_dict = torch.load(ckpt, map_location='cpu')
        self.fc.load_state_dict(state_dict)

    def predict(self, img):
        with torch.no_grad():
            logits = self.forward(img)
            return logits.sigmoid().flatten().tolist()

In [5]:
import torch.nn as nn 
import torch

from resnet_npr import resnet50

class NPR(nn.Module):
    def __init__(self):
        super(NPR, self).__init__()
        self.model = resnet50(num_classes=1)

    def forward(self, x):
        return self.model(x)

    def load_weights(self, ckpt):
        state_dict = torch.load(ckpt, map_location='cpu')
        try:
            self.model.load_state_dict(state_dict['model'], strict=False)
        except:
            print('Loading failed, trying to load model without module')
            self.model.load_state_dict(state_dict)

    def predict(self, img):
        with torch.no_grad():
            return self.forward(img).sigmoid().flatten().tolist()

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DIMD_model = DIMD().to(device)
DIMD_model.load_weights('../DIMD_latent_weights/model_epoch_best.pth')

UnivFD_model = UnivFD().to(device)
UnivFD_model.load_weights('../UnivFD_weights/fc_weights.pth')

NPR_model = NPR().to(device)
NPR_model.load_weights('../NPR_weights/NPR.pth')

/tmp/ipykernel_964832/2975036983.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(ckpt, map_location='cpu')
/tmp/ipykernel_964832/2004797838.py:1

In [12]:
model_to_test = DIMD_model

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_to_test = model_to_test.to(device)

def predict_image(image_path, model):
    img = Image.open(image_path)
    
    # Resize the image
    #img = img.resize((224, 224))
    
    # Display the image
    #plt.imshow(img)
    #plt.show()
    
    # Convert the image to a tensor and move it to the same device as the model
    img_tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)
    
    # Make prediction
    result = model.predict(img_tensor)
    return result

# Predict on single images
img_path = "cars_fake/Acura Integra Type R 2001/4.png"
img_path_2 = "cars_real/Acura Integra Type R 2001/00198.jpg"

result = predict_image(img_path, model_to_test)
result2 = predict_image(img_path_2, model_to_test)

print("Prediction for fake image: ", result)
print("Prediction for real image: ", result2)

Prediction for fake image:  [0.14770643413066864]
Prediction for real image:  [0.08515327423810959]


In [4]:
import os
from PIL import Image

def crawl_directories_and_get_image_sizes(directory):
    results = []
    max_size = (0, 0)  # Initialize max dimensions as (width, height)
    min_size = (float('inf'), float('inf'))  # Initialize min dimensions as (width, height)

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".png") or file.endswith(".jpg"):
                file_path = os.path.join(root, file)
                results.append(file_path)

                try:
                    with Image.open(file_path) as img:
                        width, height = img.size

                        # Update max size
                        if (width * height) > (max_size[0] * max_size[1]):
                            max_size = (width, height)

                        # Update min size
                        if (width * height) < (min_size[0] * min_size[1]):
                            min_size = (width, height)

                except Exception as e:
                    print(f"Error loading image {file_path}: {e}")

    return results, max_size, min_size

# Example usage
directory = "cars_real"  # Replace with your directory path
image_paths, max_dimensions, min_dimensions = crawl_directories_and_get_image_sizes(directory)
print(f"Found {len(image_paths)} images")
print(f"Max dimensions: {max_dimensions}")
print(f"Min dimensions: {min_dimensions}")

Found 64 images
Max dimensions: (2560, 1698)
Min dimensions: (220, 152)
